# Workshop - Przetwarzanie języka polskiego ze spaCy, 26 października 2020

NLPDay 2020

Ryszard Tuora


2 modele do języka polskiego w spaCy:

IPI PAN - bardziej rozbudowany, wolniejszy, bardziej złożona instalacja - http://zil.ipipan.waw.pl/SpacyPL

model oficjalny - prostszy, znacznie szybszy, prosta instalacja - https://spacy.io/models/pl

model IPI PAN dla języka polskiego

składa się z:
- taggera morfosyntaktycznego
- lematyzatora
- parsera zależnościowego
- komponentu NER
- flexera (komponentu do fleksji)

In [ ]:
# Przygotowanie środowiska, komendy linux
# instalacja Morfeusza 2
!wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
!sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
!sudo apt update
!sudo apt install morfeusz2
!sudo apt install python3-morfeusz2


# instalacja spaCy

!python3 -m pip install spacy

# 1. instalacja modelu IPI PAN dla języka polskiego
!wget "http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz"
!mv 'SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz' pl_spacy_model_morfeusz-0.1.3.tar.gz
!python3 -m pip install pl_spacy_model_morfeusz-0.1.3.tar.gz

# linkowanie modelu do spaCy
!python3 -m spacy link pl_spacy_model_morfeusz pl_spacy_model_morfeusz -f

# 2. instalacja oficjalnego modelu spaCy
!python3 -m spacy download pl_core_news_lg

# dodatkowe zależności:
!python3 -m pip install tqdm
!python3 -m pip install networkx

In [ ]:
### PYTHON 3
# ładowanie modelu
import spacy
import requests

nlp = spacy.load("pl_spacy_model_morfeusz") # IPI PAN
#nlp = spacy.load("pl_core_news_lg") # OFICJALNY

# Część zero - Tokenizacja i reprezentacja tekstów

Wejściem do modelu są łańcuchy tekstowe (stringi), na wyjściu dostajemy obiekt Doc reprezentujący struktury wykryte w tekście przez pełen potok (pipeline). Pierwszym krokiem który musi być wykonany aby przetworztć tekst, jest tokenizacja, czyli podział tekstu na tokeny/segmenty. Tokeny w większości przypadków odpowiadają słowom "od spacji do spacji". Ale warto zwrócić uwagę na kilka przypadków odstających od takiej prostej reguły.

Wynikiem potoku spaCy jest obiekt Doc, który składa się z obiektów Token.

In [ ]:
txt = "Chciałby, żebym pojechał do miasta z zielono-żółto-białą flagą (np. Zielonej Góry)."
split = txt.split()
doc = nlp(txt)
print("    spaCy     vs.   .split()\n")
for i in range(max([len(split), len(doc)])):
    try:
        tok1 = doc[i]
    except IndexError:
        tok1 = ""
    try:
        tok2 = split[i]
    except IndexError:
        tok2 = ""
    print("{0:2}. {1:15} {2:15}".format(i, tok1.orth_, tok2))

spaCy pozwala na zapisywanie przetworzonych dokumentów w formacie JSON

In [ ]:
import json

doc_json = doc.to_json()
json_string = json.dumps(doc_json, indent=2, ensure_ascii=False)
print(json_string)

# Część pierwsza - reprezentacje wektorowe

Sercem modelu jest reprezentacja wektorowa słów, 500 tys. wektorów o długości 100 wyekstrahowanych z embeddingów word2vec KGR10.

### Bardzo popularny przykład "arytmetyki słów":
znaczenie słów jest reprezentowane przez wektory, dla których mamy zdefiniowane operacje matematyczne. Możemy więc odjąć od znaczenia słowa "królowa", znaczenie słowa "kobieta", i dodać doń znaczenie słowa "mężczyzna", licząc iż wektor będący wynikiem takiego działania odpowiada słowu "król". W praktyce wektor taki najprawdopodobniej nie ma interpretacji, ale możemy znaleźć najbliższy wektor który ma jakąkolwiek interpretację przeszukując słownik.

In [ ]:
# od wersji 2.3 wektory są ładowane "leniwie"
# musimy więc najpierw wymusić załadowanie wszystkich wektorów
print(len(nlp.vocab))
for s in nlp.vocab.vectors:
    _ = nlp.vocab[s]
print(len(nlp.vocab))

In [ ]:
from scipy import spatial
 
cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)
 
man = nlp.vocab['mężczyzna'].vector
woman = nlp.vocab['kobieta'].vector
queen = nlp.vocab['królowa'].vector
king = nlp.vocab['król'].vector
 

maybe_king = man - woman + queen
computed_similarities = []
 
for word in nlp.vocab:
    # Ignore words without vectors
    if not word.has_vector:
        continue 
    similarity = cosine_similarity(maybe_king, word.vector)
    computed_similarities.append((word, similarity))
 
sorted_similarities = sorted(computed_similarities, key=lambda item: -item[1])
print([w[0].text for w in sorted_similarities[:10]])

In [ ]:
w1 = "pies"
w2 = "psami"
w3 = "zwierzę"
w4 = "buldog"
w5 = "obroża"
w6 = "smycz"
w7 = "marchewka"
w8 = "słońce"

def similarity(w1, w2):
    w1_lex = nlp.vocab[w1]
    w2_lex = nlp.vocab[w2]
    if w1_lex.has_vector and w2_lex.has_vector:
        sim = w1_lex.similarity(w2_lex)
        print("{} vs. {} -> {}".format(w1, w2, sim))
    else:
        print("Jedno ze słów nie ma reprezentacji wektorowej.")

for w in [w2, w3, w4, w5, w6, w7, w8]:
    similarity(w1, w)

## Zadanie 1:

nlp.vocab może być traktowany jako iterator: znajdują się w nim obiekty klasy lexeme, nie wszystkim obiektom lexeme przypisane są wektory - można to poznać po atrybucie .has_vector. Napisz funkcję thesaurus(word) która dla podanego słowa znajduje dziesięć najbardziej podobnych słów. Możesz skorzystać z metody .similarity zdefiniowanej dla obiektów lexeme, która jako argument przyjmuje drugi obiekt lexeme.


### spaCy umożliwia liczenie podobieństwa między słowami 
jest ono proporcjonalne do cosinusa kąta wektorami je reprezentującymi. Odpowiednia funkcja jest zdefiniowana dla leksemów (tutaj oznaczają one słowa z pominięciem kontekstu).

In [ ]:
# TODO

spaCy konstruuje także wektory dla zdań, i dokumentów. Wektor dla dokumentu jest dostępny w atrybucie .vector obiektu Doc, i jest równy uśrednionemu wektorowi tokenów z tego dokumentu.

In [ ]:
document_vector = doc.vector
mean_document_vector = sum([tok.vector for tok in doc])/len(doc)
print(document_vector == mean_document_vector)

## Dezambiguacja semantyczna

Podstawowym narzędziem służącym do określenia który z sensów wyrazu wieloznacznego występuje w tekście, jest algorytm Leska. W wersji tradycyjnej, opiera się o reprezentację BoW, i zdefiniowane dla nich miary podobieństwa, ale możemy także uogólnić go do postaci działającej dla reprezentacji word2vec.

Algorytm porównuje kontekst w jakim znajduje się docelowe słowo, z glosą, wydobytą z bazy sensów. Tutaj miarą dopasowania będzie podobieństwo cosinusowe.

## Zadanie 2:
W zmiennej wsd_data umieszczono słownik sensów dla wyrazów "zamek", "igła", "pilot", oraz "klucz", wyekstrahowany ze Słowosieci.

Przygotuj funkcję, która wykorzysta reprezentacje wektorowe zdań (lub dokumentów), do implementacji algorytmu Leska. dla każdego z przykładów, i wskazanego słowa do dezambiguacji, powinna zwrócić sens, o który chodzi w danym kontekście. Skorzystaj z metody .similarity definiowanej dla obiektów typu doc, oraz sent.


In [ ]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/workshop_resources/main/wsd_data.json").text

wsd_data = json.loads(txt)
for lemma in wsd_data:
    for variant, gloss in wsd_data[lemma]:
        print(lemma, variant, gloss)
        
sent_to_disamb1 = "Początkowo zamek miał kształt zbliżony do owalu, \
                   który był otoczony grubymi na dwa metry murami \
                   obwodowymi z blankami, na jego wewnętrznym dziedzińcu\
                   prawdopodobnie znajdowały się drewniane zabudowania."

sent_to_disamb2 = "Pielęgniarka nie mogła znaleźć żyły, igła co raz rozcinała skórę."

# TODO

# Część druga - Tagowanie morfosyntaktyczne
korzystamy z tagsetu NKJP
Nasz tagger to słownikowy tagger Morfeusz2 + dezambiguacja za pomocą neuronowego Toyggera (biLSTM)

Każdy token t ma trzy interesujące nas atrybuty: 
- t.tag_ : klasa gramatyczna według polskiego tagsetu NKJP (http://nkjp.pl/poliqarp/help/ense2.html)
- t.pos_ : klasa gramatyczna według międzynarodowego tagsetu UD (mapowana z NKJP)
- t._.feats : customowy atrybut odpowiadający cechom morfosyntaktycznym (np. rodzajowi gramatycznemu, lub liczbie), poszczególne wartości cech są oddzielone dwukropkiem

In [ ]:
txt = "Nornica prowadzi zmierzchowo-nocny tryb życia, ale wychodzi również za dnia w poszukiwaniu pokarmu."
doc = nlp(txt) # przetworzenie textu przez pipeline, na wyjściu dostajemy iterowalny obiekty klasy Doc, przechowujący tokeny

print("{0:15} {1:8} {2:6} {3:15}\n".format(".orth_", "NKJP", "UD POS", "._.feats"))
for t in doc:
    print("{0:15} {1:8} {2:6} {3:15}".format(t.orth_, t.tag_, t.pos_, t._.feats)) # wypisujemy interpretację morfosyntaktyczną każdego tokenu

## Zadanie 3:

Charakterystyka stylów literackich wiąże się z proprocjami części mowy w tekście.

![alt text](https://github.com/ryszardtuora/workshop_resources/raw/main/czesci_mowy.png)

Źródło: Irena Kamińska-Szmaj, *Różnice leksykalne między stylami funkcjonalnymi polszczyzny pisanej: Analiza statystyczna na materiale słownika frekwencyjnego*, 1990.

Przetwórz tekst znajdujący się w zmiennej txt, oblicz proporcję czasowników (używając tagów UD), na podstawie tego oszacuj gatunek do którego należy tekst.

In [ ]:
import requests

txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/workshop_resources/main/1.txt").text

# TODO

### Fleksja

Flexer pozwala na odmianę pojedynczych tokenów, do pożądanej charakterystyki morfologicznej. Argumentami do flexera jest słowo do odmiany (string, lub lepiej otagowany token), i przedzielony dwukropkami string znaczników morfosyntaktycznych.

Flexer umożliwia np. wypełnianie szablonów tekstów.

In [ ]:
filizanka = "filiżanka"
flexer = nlp.get_pipe("flexer")

tmpl1 = "Szukam szarej, ceramicznej {}.".format(flexer.flex(filizanka, "gen"))
tmpl2 = "Marzę o szarej, ceramicznej {}.".format(flexer.flex(filizanka, "loc"))
tmpl3 = "Szukam kompletu ceramicznych {}.".format(flexer.flex(filizanka, "gen:pl"))
print(tmpl1)
print(tmpl2)
print(tmpl3)

#### Flexer pozwala także na odmianę fraz wielowyrazowych (multi-word expressions - MWE).

W tym celu możemy do niego podać string, lub token odpowiadający głowie frazy.
Korzystamy z metody .flex_mwe()

In [ ]:
filizanka2 = "biała, porcelanowa filiżanka z Chin"

tmpl1 = "Szukam {}.".format(flexer.flex_mwe(filizanka2, "gen"))
tmpl2 = "Marzę o {}.".format(flexer.flex_mwe(filizanka2, "loc"))
tmpl3 = "Szukam kompletu {}.".format(flexer.flex_mwe(filizanka2, "gen:pl"))

print(tmpl1)
print(tmpl2)
print(tmpl3)

## Zadanie 4

Zaprojektuj sposób formułowania znaczników morfosyntaktycznych dla luk w szablonach, który określa jaką formę muszą przyjąć frazy, by móc wypełnić te luki.
Zaprojektuj funkcję, która pobiera jako argumenty 1. szablon z tak oznakowaną luką, 2. frazę do wypełnienia luki, która odmienia wskazaną frazę, i wkleja ją w miejsce luki. 
Możesz założyć, że funkcja ma działać wtedy, i tylko wtedy, gdy w szablonie znajduje się dokładnie jedna luka.

In [ ]:
template = "Przyglądam się ___." # w miejsce ___ wstaw zaprojektowany znacznik, który pozwoli na poprawne
                                 # uzupełnienie luki
phrase = "czarne, skórzana torebka ze sprzączką"
#TODO

#### Do modelu można załadować także customowy słownik Morfeusza, wzbogacony o słownictwo dziedzinowe.
W tym celu należy pobrać komponent tokenizatora (nlp.tokenizer) oraz flexer, i utworzyć dla nich nową instancję Morfeusza.

In [ ]:
!wget https://github.com/ryszardtuora/workshop_resources/blob/main/cust_dict.tar.gz?raw=true
!mv 'cust_dict.tar.gz?raw=true' cust_dict.tar.gz
!tar -xvf cust_dict.tar.gz

In [ ]:
import morfeusz2

flexer = nlp.get_pipe("flexer")
print("Przed: ", flexer.flex("sneakersy", "gen"))

dict_name = "cust_dict"
dict_path = "cust_dict"
morf = morfeusz2.Morfeusz(whitespace = morfeusz2.KEEP_WHITESPACES,
                                   expand_tags=True,
                                   dict_name=dict_name, dict_path=dict_path)
tokenizer = nlp.tokenizer
tokenizer.morf = morf
flexer.morf = morf
print("Po:", flexer.flex("sneakersy", "gen"))

# zresetujmy model
nlp = spacy.load("pl_spacy_model_morfeusz")

# Część trzecia - Lematyzacja i własności leksykalne
Nasz model umożliwia słownikową lematyzację przy pomocy Morfeusza, do dezambiguacji (tutaj np. rozróżnienia między "Głos zabrały mamy dzieci."-> "mama" i "My mamy samochód." -> "mieć" służy output taggera).

Lematyzacja pozwala redukować redundancję informacyjną, i ułatwiać zadania takie jak streszczanie, i przeszukiwanie.

Każdy z tokenów dodatkowo jest oznaczony ze względu na pewne własności leksykalne, np. :
- t.lemma_ - lemat - forma reprezentatywna danego leksemu
- t.is_stop - słowo należy do stoplisty (listy słów występujących najczęściej, a więc najmniej istotnych semantycznie)
- t.is_oov - słowo znajduje się poza słownikiem, i.e. embeddingami wykorzystanymi w modelu
- t.like_url - token ma strukturę url-a
- t.like_num - token jest liczbą
- t.is_alpha - token składa się tylko ze znaków alfabetycznych
- t.rank - miejse w rankingu częstości słów

In [ ]:
txt = "Zdenerwowany gen. Leese mówił przez telefon swym podwładnym walczącym pod Monte Cassino, że rozmawia z nimi ze schronu."
doc = nlp(txt)
print("{0:16} {1:16} {2:5} {3:5} {4:20}\n".format("forma", "lemat", "OOV", "STOP", "Częstość"))
for t in doc:
    print("{0:16} {1:16} {2:5} {3:5} {4:20}".format(t.orth_, t.lemma_, t.is_oov, t.is_stop, t.rank)) # orth_ to atrybut odpowiadający formie słowa występującej w tekście

## Zadanie 5:

1. Przetwórz tekst spod zmiennej txt, 
2. Przekonwertuj go do listy lematów
3. usuń słowa ze stoplisty, oraz interpunkcję, 
4. wypisz dziesięć najczęściej pojawiających się lematów. 

Wypróbuj także opcję w której uwzględniamy tylko rzeczowniki.


In [ ]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/workshop_resources/main/2.txt").text

#TODO

# Część czwarta - parsowanie zależnościowe

spaCy zawiera parser zależnościowy oparty o metodologię transition-based dependency parsing. 
Interesują nas tu cztery atrybuty:
 - t.head - link do tokenu będącego nadrzędnikiem tokenu t
 - t.dep_ - etykieta opisująca rodzaj zależności
 - t.subtree - generator opisujący poddrzewo rozpięte przez token t
 - t.children - generator opisujący wszystkich bezpośrednich potomków tokenu t
 - t.ancestors - generator opisujący wszystkie przechodnie nadrzędniki tokenu t


Opis systemu etykiet: https://universaldependencies.org/u/dep/all.html

In [ ]:
txt = "Pierwsza wzmianka o Gdańsku pochodzi ze spisanego po łacinie w 999 Żywotu świętego Wojciecha."
doc = nlp(txt)

import pandas as pd

table = []
for tok in doc:
    tok_dic = {"form": tok.orth_, "label": tok.dep_, "head": tok.head.orth_, "subtree": list(tok.subtree), "ancestors": list(tok.ancestors)}
    table.append(tok_dic)
df = pd.DataFrame(table)
print(df.to_string())

### spaCy posiada wbudowaną wizualizację drzew zależnościowych

In [ ]:
from spacy import displacy

displacy.render(doc, jupyter = True)

### Poniższa funkcja służy łatwej wizualizacji podstawowych własności tokenów z danego tekstu

In [ ]:
import pandas as pd

def table(doc):
    table = []
    for tok in doc:
        tok_dic = {"form": tok.orth_, "lemma": tok.lemma_, "tag": ":".join([tok.tag_, tok._.feats]), "dep_label": tok.dep_, "dep_head": tok.head.orth_}
        table.append(tok_dic)
    return pd.DataFrame(table)

txt = "Wiadomość jest symboliczna, ale oznacza też początek długotrwałego trendu.\
 Dochód na mieszkańca z uwzględnieniem realnej mocy nabywczej walut narodowych \
 wyniósł w 2019 r. w Rzeczpospolitej Polskiej 33 891 dolarów, nieco więcej, niż w Portugalii \
 (33 665 dolarów). Jednak Fundusz przewiduje, że w tym roku portugalska gospodarka\
  będzie się rozwijać w tempie 1,6 proc. wobec 3,1 proc. w przypadku gospodarki \
  polskiej. Nożyce między oboma krajami będą się więc rozwierać."

doc = nlp(txt)

tab = table(doc)

print(tab.to_string()) # prosty hack na wypisanie całej tabeli

### Czasami interesują nas większe całości niż pojedyncze tokeny, 
np. rzeczowniki często łączą się w frazy rzeczownikowe, żeby znajdować takie całości w tekście możemy korzystać z Matchera.

In [ ]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
pattern = [{"POS": "NOUN"}, {"POS": "ADP"}, {"POS": "NOUN"}]
matcher.add("NounAdpNoun", None, pattern) # nazwa, funkcja, wzór

for match_id, start, end in matcher(doc):
    toks = doc[start:end]
    print(toks) # obiekt typu Span

### Lematyzacja jednostek wielowyrazowych

Flexer pozwala także na prostą lematyzację jednostek wielowyrazowych.

In [ ]:
print("Flexer vs. konkatenacja lematów:\n")
for match_id, start, end in matcher(doc):
    phrase = doc[start:end]
    phrase_text = phrase.text
    toks = [t for t in phrase]
    lemmas = [t.lemma_ for t in toks]
    lemma_concat = " ".join(lemmas)
    print("{} vs. {}".format(flexer.lemmatize_mwe(phrase_text), lemma_concat))



## Zadanie 6

Znajdź wszystkie frazy postaci przymiotnik - rzeczownik, i wypisz je w formie występującej w tekście, oraz formie zlematyzowanej.


In [ ]:
# TODO

#### Matcher pozwala także na korzystanie z innych własności tokenów, np. lematów.

### Parser zależnościowy pozwala także na dzielenie dokumentów na zdania w sposób bardziej inteligentny, niż posługując się regułami interpunkcji.
Za zdanie uważamy nieprzerwaną sekwencję tokenów które są powiązane relacjami zależnościowymi.
Zdania są zapisane w atrybucie doc.sents dokumentu.

In [ ]:
for s in doc.sents:
    print(s)

displacy.render(doc, jupyter = True)

# Część czwarta - Rozpoznawanie jednostek nazewniczych (NER)
#### Nasz model do spaCy wykorzystuje 6 rodzajów etykiet:
- placeName - miejsca antropogeniczne, np. Dania, Londyn
- geogName - naturalne miejsca geograficzne, np. Tatry, Kreta
- persName - imiona i nazwiska osób, np. J. F. Kennedy, gen. Maczek
- orgName - nazwy organizacji, np. NATO, Unia Europejska
- date - daty, np. 22 marca 1999, druga połowa kwietnia
- time - czas, np. 18:55, pięć po dwunastej

#### Nie pozwala na wykrywanie zagnieżdżonych jednostek nazewniczych, np. [placeName: **aleja** [persName: **Piłsudskiego**]]

Wykryte wzmianki są przechowywane w atrybucie doc.ents dokumentu, każda z tych wzmianek ma atrybut ent.label_, w którym przechowywana jest jej etykieta.

In [ ]:
print(doc, "\n\n")
for e in doc.ents:
    print(e, e.label_)


### displaCy pozwala także na wizualizację jednostek nazewniczych

In [ ]:
displacy.render(doc, style="ent", jupyter = True)

W obecnym modelu NER, uwzględnione są "uniwersalne" kategorie jednostek nazewniczych, jednak w zależności od zastosowania będziemy najprawdopodobniej potrzebowali innych kategorii - np. osobnej kategorii dla nazw aktów prawodawczych, lub kwot i walut.

Bez problemu można zastąpić domyślny model własnym, wytrenowanym przez CLI spaCy na własnych oznakowanych danych. Więcej informacji tu: https://spacy.io/api/cli#train

Jednostki które da się wykryć na podstawie reguł, można wykrywać poprzez EntityRuler: https://spacy.io/api/entityruler

## Zadanie 7:

W zmiennej txt znajduje się dokument historyczny, wydobądź z niego wszystkie *zdania* zawierające daty, i po kolei je wypisz, rozważ możliwe sposoby automatycznego szeregowania dat wyrażonych w różny sposób (po ewentualnym sprowadzeniu ich do kanonicznej postaci). Rozwiązanie tego problemu jest trudne, i istnieją do niego odrębne narzędzia, umożliwiają one np. automatyczną rekonstrukcję chronologii wydarzeń.

In [ ]:
# TODO

## Zadanie 8:

W zmiennej txt znajduje się dokument zawierający nazwiska ludzi. Wykryj pojawiające się w nim imiona i nazwiska osób, i zastąp je znacznikiem - tj. poddaj tekst anonimizacji.

In [ ]:
pers_marker = "@Anonymized@"

txt = """26-latek trzy lata temu jako piłkarz Levante UD strzelił Realowi Madryt gola na Santiago Bernabeu, a dziś wyrasta na jednego z liderów zespołu Marka Papszuna.
Lopez w meczu z beniaminkiem najpierw dołożył w polu karnym nogę do podania Marko Poletanovicia, a później pięknie przymierzył z rzutu wolnego. 
Raków zwyciężył, choć na ławce spotkanie rozpoczął Czech Petr Schwarz, a w ogóle nie zagrał Marcin Cebula, czyli bohaterowie pierwszych kolejek.
"""

#TODO

# Część Piąta - Zastosowania Różne

W tej części pokażemy kilka przykładów wykorzystania spaCy razem z innymi bibliotekami

### Streszczanie

Streszczanie w najprostszej formie polega na wybraniu najistotniejszych zdań z dokumentu.
W materiałach znajduje się zescrapowany z Wikipedii artykuł nt. Polityki Gospodarczej, długości 92 zdań.

In [ ]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/workshop_resources/main/polityka_gospodarcza.txt").text

doc = nlp(txt)
sents = list(doc.sents)
n_sents = len(sents)
print(n_sents)

### Jednym ze sposobów punktowania zdań jest wykorzystanie algorytmu PageRank

W tym celu musimy zinterpretować macierz podobieństwa zdań w dokumencie, jako macierz sąsiedztwa w grafie. Podobieństwo zaś, liczymy jako podobieństwo cosinusowe zdań do siebie.

In [ ]:
import numpy
from sklearn.metrics import pairwise
import networkx as nx

TOP_N = 5

# przygotowanie macierzy podobieństwa
vex = numpy.array([sent.vector for sent in sents])
sim_matrix = 1 - pairwise.cosine_distances(vex, vex)

# zastosowanie algorytmu TextRank
sim_graph = nx.from_numpy_matrix(sim_matrix)
sent_scores = nx.pagerank(sim_graph)


# przygotowanie wyników
ranked_sents = sorted(((sent_scores[i], i, sent) for i, sent in enumerate(sents)), reverse=True)
top_n = ranked_sents[:TOP_N]
ordered_top_n = sorted(top_n, key=lambda x:x[1])
summary_sents = [e[2].text for e in ordered_top_n]
summary = " ".join(summary_sents)
print(summary)

#### Aby zobaczyć co pominęliśmy, możemy wykorzystać displacy, i zaznaczyć nasze zdania jako entities, podobne do tych, wykrywanych przez NER.
W tym celu utworzymy obiekty klasy Span, i skorzystamy z atrybutów start i end

In [ ]:
from spacy.tokens import Span

ents = []
for score, index, sent in top_n:
    start = sent.start
    end = sent.end
    label = "SUMM"
    span = Span(doc, start, end, label)
    ents.append(span)
doc.ents = ents # nadpisywanie wykrytych w tekście obiektów
displacy.render(doc, style="ent", jupyter = True)

## Zadanie 9:

Zmierz stosunek długości streszczenia względem tekstu pierwotnego. Poeksperymentuj z parametrem TOP_N i oszacuj, dla jakiej wielkości uzyskujemy najlepsze streszczenie.

In [ ]:
# TODO

### Zadanie 10:
#### Wraz z wzrostem liczby użytych zdań, wzrasta ryzyko pojawienia się redundancji

Zastanów się w jaki sposób możemy przefiltrować wybrane zdania, aby uniknąć powtórzeń?

In [ ]:
#TODO


In [ ]:
# przygotowanie wyników
ranked_sents = sorted(((sent_scores[i], i, sent) for i, sent in enumerate(sents)), reverse=True)
top_n = ranked_sents[:TOP_N]

#filtrowanie przez klastry
sents_by_clusters = {}
for score, i, sent in top_n:
    label = int(labels[i])
    if label not in sents_by_clusters:
        sents_by_clusters[label] = (i, sent)
    else:
        if len(sent) > len(sents_by_clusters[label][1]):
            sents_by_clusters[label] = (i, sent)
filtered = [sents_by_clusters[label] for label in sents_by_clusters]
ordered = sorted(filtered)
summary_sents = [e[1].text for e in ordered]
summary = " ".join(summary_sents)
print(len(summary_sents))
print(summary)
            

## Klasyfikacja tekstów

Jeżeli chcemy skonstruować potok do klasyfikacji tekstów, spaCy może spełnić w nim różne role.
  1. może być źródłem embeddingów
  2. może służyć do preprocessingu i czyszczenia danych (tokenizacji, filtrowania, reprezentacji wektorowej)

Ale:
spaCy ma także wbudowany komponent klasyfikatora tekstów, który możemy sami wyuczyć na naszych danych.

In [ ]:
!wget https://github.com/ryszardtuora/workshop_resources/blob/main/poleval.tar.gz?raw=true
!mv 'poleval.tar.gz?raw=true' poleval.tar.gz
!tar -xvf poleval.tar.gz

In [ ]:
import random



def load_data():
    off_train = 0
    off_test = 0
    with open("poleval/training_set_clean_only_text.txt") as f:
        txt = f.read()
        train_sents = txt.split("\n")[:-1]
    with open("poleval/training_set_clean_only_tags.txt") as f:
        txt = f.read()
        train_labels = [int(x) for x in txt.split("\n")[:-1]]
    with open("poleval/Task6/task 01/test_set_clean_only_text.txt") as f:
        txt = f.read()
        test_sents = txt.split("\n")[:-1]
    with open("poleval/Task6/task 01/test_set_clean_only_tags.txt") as f:
        txt = f.read()
        test_labels = [int(x) for x in txt.split("\n")[:-1]]

    # oversampling
    off_sents = []
    for x, y in zip(train_sents, train_labels):
        if y == 1:
            off_sents.append(x)
    oversampling_scale = 8
    oversampling_x = oversampling_scale * off_sents
    oversampling_y = [1 for x in oversampling_x]
    train_sents.extend(oversampling_x)
    train_labels.extend(oversampling_y)
    indices = list(range(len(train_sents)))
    random.shuffle(indices)
    train_sents = [train_sents[i] for i in indices]
    train_labels = [train_labels[i] for i in indices]

    train_cats = []
    test_cats = []
    
    ### Przygotowanie anotacji danych dla spaCy
    # Anotacje mają postać listy słowników z wartościami logicznymi dla każdej z etykiet
    for tl in train_labels:
        train_cats.append({"OFF": bool(tl), "NONOFF": not bool(tl)})
        if tl:
            off_train += 1
    for tl in test_labels:
        test_cats.append({"OFF": bool(tl), "NONOFF": not bool(tl)})
        if tl:
            off_test += 1
    ###
    print("Proportion of offensive tweets in the train data: {}".format(off_train/len(train_sents)))
    print("Proportion of offensive tweets in the test data: {}".format(off_test/len(test_sents)))
    return train_sents, test_sents, train_cats, test_cats

train_sents, test_sents, train_cats, test_cats = load_data()
print("Data loaded!")


In [ ]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if label == "NONOFF":
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}



In [ ]:
from spacy.util import minibatch, compounding

nlp = spacy.load("pl_spacy_model_morfeusz")
# Inicjalizacja modelu
textcat = nlp.create_pipe("textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})
nlp.add_pipe(textcat, last=True)
textcat.add_label("OFF")
textcat.add_label("NONOFF")

# Trening
n_texts = 1000
n_iter = 2
train_sents = train_sents[:n_texts]
train_cats = train_cats[:n_texts]
train_data = list(zip(train_sents, [{"cats": cats} for cats in train_cats]))

print(len(train_sents))

pipe_exceptions = ["textcat"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()
    print("Training the model...")
    print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
    batch_sizes = compounding(4.0, 32.0, 1.001)
    for i in range(n_iter):
        losses = {}
        # batch up the examples using spaCy's minibatch
        random.shuffle(train_data)
        batches = minibatch(train_data, size=batch_sizes)
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
        with textcat.model.use_params(optimizer.averages):
            scores = evaluate(nlp.tokenizer, textcat, test_sents, test_cats)
        print("{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format(  # print a simple table
                losses["textcat"],
                scores["textcat_p"],
                scores["textcat_r"],
                scores["textcat_f"],
            )
        )


#### Wykorzystanie wytrenowanego modelu:

In [ ]:
doc = nlp("Powieś się debilu.")
print(doc.cats)